<a href="https://colab.research.google.com/github/kavithachitriki/OCRProject/blob/main/KerasRetinanet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [7]:
!git clone https://github.com/fizyr/keras-retinanet

Cloning into 'keras-retinanet'...
remote: Enumerating objects: 6205, done.
remote: Total 6205 (delta 0), reused 0 (delta 0), pack-reused 6205
Receiving objects: 100% (6205/6205), 13.47 MiB | 27.25 MiB/s, done.
Resolving deltas: 100% (4201/4201), done.


In [11]:
%cd keras-retinanet/
!pip install .

[Errno 2] No such file or directory: 'keras-retinanet/'
/content/keras-retinanet
Processing /content/keras-retinanet
  Created wheel for keras-retinanet: filename=keras_retinanet-1.0.0-cp37-cp37m-linux_x86_64.whl size=168008 sha256=bf7a64643fbbb1934eec72a3a1a3993a36b3bb476e47ca2b5bbe954def276b91
  Stored in directory: /root/.cache/pip/wheels/b2/9f/57/cb0305f6f5a41fc3c11ad67b8cedfbe9127775b563337827ba
  Created wheel for keras-resnet: filename=keras_resnet-0.2.0-py2.py3-none-any.whl size=20486 sha256=9c5b4c255b49a91cd334a2ca626f15998ba2e94bc0974b708db1ea4aea74c363
  Stored in directory: /root/.cache/pip/wheels/5f/09/a5/497a30fd9ad9964e98a1254d1e164bcd1b8a5eda36197ecb3c
Successfully built keras-retinanet keras-resnet


In [12]:
!python setup.py build_ext --inplace

running build_ext
cythoning keras_retinanet/utils/compute_overlap.pyx to keras_retinanet/utils/compute_overlap.c
/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/keras-retinanet/keras_retinanet/utils/compute_overlap.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'keras_retinanet.utils.compute_overlap' extension
creating build
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/keras_retinanet
creating build/temp.linux-x86_64-3.7/keras_retinanet/utils
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-a56wZI/python3.7-3.7.10=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-a56wZI/python3.7-3.7.10=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -

In [13]:
!pip install gdown
!pip install tensorflow.gpu

     |████████████████████████████████| 394.3MB 36kB/s 


In [17]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters
from sklearn.model_selection import train_test_split
import urllib
import os
import csv
import cv2
import time
from PIL import Image

from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr,preprocess_image,resize_image
from keras_retinanet.utils.visualization import draw_box,draw_caption
from keras_retinanet.utils.colors import label_color

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [18]:
register_matplotlib_converters()
sns.set(style='whitegrid',palette='muted',font_scale=1.5)

rcParams['figure.figsize'] = 22,10

RANDOM_SEED=42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [20]:
os.makedirs("snapshots",exist_ok=True)

In [21]:
!gdown --id 1wPgOBoSks6bTIs9RzNvZf6HWROkciS8R --output snapshots/resnet50_csv_10.h5

Downloading...
From: https://drive.google.com/uc?id=1wPgOBoSks6bTIs9RzNvZf6HWROkciS8R
To: /content/keras-retinanet/snapshots/resnet50_csv_10.h5
249MB [00:02, 107MB/s]


In [23]:
dataset=dict()
dataset["image_name"]=list()
dataset["x_min"]=list()
dataset["y_min"]=list()
dataset["x_max"]=list()
dataset["y_max"]=list()
dataset["class_name"]=list()

counter=0

In [34]:
#creating dataset
#dataset["image_name"].append("")
img1="https://drive.google.com/file/d/1YtDG1FdLienKm5I-n5vwF2Frslsk6-Rj/view?usp=sharing"
img2="https://drive.google.com/file/d/1hYujwJlDNHsrtDjTSllDFfiqwQx15pdK/view?usp=sharing"
img3="https://drive.google.com/file/d/1fRKNgE0eYCJvjw1XWTJzn8K071f4HQuG/view?usp=sharing"
img4="https://drive.google.com/file/d/1nXYCND08BXrcW5ieZsX4NJQQN3UwBagm/view?usp=sharing"

counter=0
#img=urllib.request.urlopen('pan1.jpeg')
img=Image.open('../pan1.jpeg').convert('RGB')
img.save(f'../panimages/pan{counter}.jpeg','JPEG')
dataset["image_name"].append(f'panimages/pan{counter}.jpeg')

counter=1
img=Image.open('../pan2.jpeg').convert('RGB')
img.save(f'../panimages/pan{counter}.jpeg','JPEG')
dataset["image_name"].append(f'panimages/pan{counter}.jpeg')

counter=2
img=Image.open('../pan3.jpeg').convert('RGB')
img.save(f'../panimages/pan{counter}.jpeg','JPEG')
dataset["image_name"].append(f'panimages/pan{counter}.jpeg')

counter=3
img=Image.open('../pan4.jpeg').convert('RGB')
img.save(f'../panimages/pan{counter}.jpeg','JPEG')
dataset["image_name"].append(f'panimages/pan{counter}.jpeg')


